# Class Workbook

## In class activity

In [1]:
import numpy as np
import pandas as pd
import math
from matplotlib.pyplot import subplots
#import statsmodels.api as sm
from plotnine import *
import plotly.express as px
import statsmodels.formula.api as sm
#import ISLP as islp

import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.linear_model import LinearRegression

import torch
from torch import nn
from torch.optim import RMSprop
import torch.optim as optim
from torch.utils.data import TensorDataset
from sklearn.preprocessing import StandardScaler

### Ames Housing data


Please take a look at the Ames Hoursing data.

In [2]:
ames_raw=pd.read_csv("ames_raw.csv")

Use data of `ames_raw` up to 2008 predict the housing price for the later years.

In [3]:
ames_raw_2009, ames_raw_2008= ames_raw.query('`Yr Sold`>=2008').copy(), ames_raw.query('`Yr Sold` <2008').copy()

In [4]:
ames_raw_2008.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1319 entries, 1611 to 2929
Data columns (total 82 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order            1319 non-null   int64  
 1   PID              1319 non-null   int64  
 2   MS SubClass      1319 non-null   int64  
 3   MS Zoning        1319 non-null   object 
 4   Lot Frontage     1106 non-null   float64
 5   Lot Area         1319 non-null   int64  
 6   Street           1319 non-null   object 
 7   Alley            84 non-null     object 
 8   Lot Shape        1319 non-null   object 
 9   Land Contour     1319 non-null   object 
 10  Utilities        1319 non-null   object 
 11  Lot Config       1319 non-null   object 
 12  Land Slope       1319 non-null   object 
 13  Neighborhood     1319 non-null   object 
 14  Condition 1      1319 non-null   object 
 15  Condition 2      1319 non-null   object 
 16  Bldg Type        1319 non-null   object 
 17  House Style     

Use the following loss function calculator.

In [5]:
def calc_loss(prediction,actual):
  difpred = actual-prediction
  RMSE =pow(difpred.pow(2).mean(),1/2)
  operation_loss=abs(sum(difpred[difpred<0]))+sum(0.1*actual[difpred>0])
  return RMSE,operation_loss

Use a simple neural network model.

In [9]:
# Define your neural network architecture
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(16, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define target column
inputs = ames_raw_2008[['Lot Area', 'Overall Qual', 'Overall Cond', 'Year Built', 
                        'Year Remod/Add', '1st Flr SF', '2nd Flr SF', 'Gr Liv Area',
                        'Full Bath', 'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr',
                        'TotRms AbvGrd', 'Pool Area', 'Mo Sold', 'Yr Sold']]
targets = ames_raw_2008['SalePrice']

# Standardize inputs and targets
input_scaler = StandardScaler()
inputs = input_scaler.fit_transform(inputs)

target_scaler = StandardScaler()
targets = target_scaler.fit_transform(targets.values.reshape(-1, 1))

# Convert to PyTorch tensors
inputs = torch.tensor(inputs, dtype=torch.float32)
targets = torch.tensor(targets, dtype=torch.float32)

# Create an instance of your neural network
nnfit_2008 = NeuralNetwork()

# Define your loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(nnfit_2008.parameters(), lr=0.01)

# Perform training
for epoch in range(100):
    # Forward pass
    outputs = nnfit_2008(inputs)
    loss = criterion(outputs, targets)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print progress
    if (epoch+1) % 10 == 0:
        print(f'Epoch: {epoch+1}, Loss: {loss.item()}')

Epoch: 10, Loss: 0.952707052230835
Epoch: 20, Loss: 0.8708751797676086
Epoch: 30, Loss: 0.7827321290969849
Epoch: 40, Loss: 0.6860052347183228
Epoch: 50, Loss: 0.5846449136734009
Epoch: 60, Loss: 0.48753291368484497
Epoch: 70, Loss: 0.40397322177886963
Epoch: 80, Loss: 0.33894506096839905
Epoch: 90, Loss: 0.29235079884529114
Epoch: 100, Loss: 0.26089081168174744


When you decide on your model use the following to come up with your test loss.

In [13]:
# Select the same input columns
inputs_2009 = ames_raw_2009[['Lot Area', 'Overall Qual', 'Overall Cond', 'Year Built', 
                             'Year Remod/Add', '1st Flr SF', '2nd Flr SF', 'Gr Liv Area',
                             'Full Bath', 'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr',
                             'TotRms AbvGrd', 'Pool Area', 'Mo Sold', 'Yr Sold']]

# Standardize the inputs using the same scaler
inputs_2009 = input_scaler.transform(inputs_2009)

# Convert to PyTorch tensors
inputs_2009 = torch.tensor(inputs_2009, dtype=torch.float32)

# Use the trained model to make predictions
outputs_2009 = nnfit_2008(inputs_2009)

# Convert the predictions back to the original scale
pred_2009 = target_scaler.inverse_transform(outputs_2009.detach().numpy())

# Convert pred_2009 to a one-dimensional array
pred_2009 = pred_2009.ravel()

# Predict using ames_raw_2009
calc_loss(pred_2009,ames_raw_2009.SalePrice)

(43129.36117382642, 46816852.18125)

Try to answer the following additional questions.

- Does your model indicate a good fit?


- How does your model result compare to the previous models you fit?


- Can you explain what feature was important determinant of the price?

### COVID 19 Survival in Mexico

Let's revisit COVID-19 in Mexico dataset from the [Mexican government](https://datos.gob.mx/busca/dataset/informacion-referente-a-casos-covid-19-en-mexico).  This data is a version downloaded from [Kaggle](https://www.kaggle.com/datasets/meirnizri/covid19-dataset?resource=download).  The raw dataset consists of 21 unique features and 1,048,576 unique patients. In the Boolean features, 1 means "yes" and 2 means "no". values as 97 and 99 are missing data.

- sex: 1 for female and 2 for male.
- age: of the patient.
- classification: COVID test findings. Values 1-3 mean that the patient was diagnosed with COVID in different degrees. 4 or higher means that the patient is not a carrier of COVID or that the test is inconclusive.
- patient type: type of care the patient received in the unit. 1 for returned home and 2 for hospitalization.
- pneumonia: whether the patient already have air sacs inflammation or not.
- pregnancy: whether the patient is pregnant or not.
- diabetes: whether the patient has diabetes or not.
- copd: Indicates whether the patient has Chronic obstructive pulmonary disease or not.
- asthma: whether the patient has asthma or not.
- inmsupr: whether the patient is immunosuppressed or not.
- hypertension: whether the patient has hypertension or not.
- cardiovascular: whether the patient has heart or blood vessels related disease.
- renal chronic: whether the patient has chronic renal disease or not.
- other disease: whether the patient has other disease or not.
- obesity: whether the patient is obese or not.
- tobacco: whether the patient is a tobacco user.
- usmr: Indicates whether the patient treated medical units of the first, second or third level.
- medical unit: type of institution of the National Health System that provided the care.
- intubed: whether the patient was connected to the ventilator.
- icu: Indicates whether the patient had been admitted to an Intensive Care Unit.
- date died: If the patient died indicate the date of death, and 9999-99-99 otherwise.

In [ ]:
import zipfile
Train_COVID= pd.read_csv('Train_COVID.zip',compression='zip')
Test_COVID= pd.read_csv('Test_COVID.zip',compression='zip')

- Fit a sequence model that predicts the number of cases a week a head.

- Modify your model to make prediction for different gender.


Your code:

Your answer:

~~~
Please write your answer in full sentences.


~~~

## Problem set

### Writing your own gradient decent

Consider the simple function $R(\beta) = sin(\beta) + \beta/10$.

(a) Draw a graph of this function over the range $\beta \in [−6, 6]$.
Your code:

In [ ]:
#
#

(b) What is the derivative of this function?

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

(c) Given $\beta_0 = 2.3$, run gradient descent to find a local minimum of $R(\beat)$ using a learning rate of $\rho= 0.1$. Show each of $\beta_0,\beta_1,\dots$ in your plot, as well as the final answer.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

(d) Repeat with $\beta_0 = 1.4$.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

### Default

Fit a neural network to the Default data. Use a single hidden layer with 10 units, and dropout regularization. Have a look at Labs 10.9.1–10.9.2 for guidance. Compare the classification performance of your model with that of linear logistic regression.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

### IMDb

Repeat the analysis of Lab 10.9.5 on the IMDb data using a similarly structured neural network. We used 16 hidden units at each of two hidden layers. Explore the effect of increasing this to 32 and 64 units per layer, with and without 30% dropout regularization.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

### NYSE

Fit a lag-5 autoregressive model to the NYSE data, as described in the text and Lab 10.9.6. Refit the model with a 12-level factor representing the month. Does this factor improve the performance of the model?


Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

### NYSE 2
In Section 10.9.6, we showed how to fit a linear AR model to the
NYSE data using the `LinearRegression()` function. However, we also
mentioned that we can “flatten” the short sequences produced for
the RNN model in order to fit a linear AR model. Use this latter
approach to fit a linear AR model to the NYSE data. Compare the test
R2 of this linear AR model to that of the linear AR model that we fit
in the lab. What are the advantages/disadvantages of each approach?

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

Repeat the previous exercise, but now fit a nonlinear AR model by
“flattening” the short sequences produced for the RNN model.

 Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

### NYSE 3

Consider the RNN fit to the NYSE data in Section 10.9.6. Modify the code to allow inclusion of the variable day_of_week, and fit the RNN. Compute the test $R^2$.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

### CNN on photo

From your collection of personal photographs, pick 10 images of animals
(such as dogs, cats, birds, farm animals, etc.). If the subject
does not occupy a reasonable part of the image, then crop the image.
Now use a pretrained image classification CNN as in Lab 10.9.4 to
predict the class of each of your images, and report the probabilities
for the top five predicted classes for each image.

Your code:

In [5]:
from torchvision import models
import torch
from torchvision.models import alexnet
from torchvision.models import AlexNet_Weights
 
weights=AlexNet_Weights.IMAGENET1K_V1
alexnet_model= alexnet(weights=weights)
print(alexnet_model)
 
# You can also do the same by down laoding from torch.hub
# AlexNet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)
 
from torchvision import transforms
transform = transforms.Compose([            #[1]
transforms.Resize(256),                   #[2]
transforms.CenterCrop(224),               #[3]
transforms.ToTensor(),                    #[4]
transforms.Normalize(                      #[5]
mean=[0.485, 0.456, 0.406],               #[6]
std=[0.229, 0.224, 0.225]                  #[7]
)])

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [6]:
import os
currentdir=os.getcwd()
os.chdir('images')
files=os.listdir(".")
 
# Import Pillow
from PIL import Image

image_extensions = ['.jpg', '.png', '.jpeg', '.bmp', '.gif']

for file in files:
  if os.path.splitext(file)[1].lower() in image_extensions:
    img = Image.open(file)

    img_t = transform(img)
    batch_t = torch.unsqueeze(img_t, 0)

    alexnet_model.eval()

    out = alexnet_model(batch_t).squeeze(0).softmax(0)
    class_id = out.argmax().item()
    score = out[class_id].item()
    category_name = weights.meta["categories"][class_id]
    print(f"File: {file}, Category: {category_name}, Score: {100 * score:.1f}%")
 
os.chdir(currentdir)

File: IMG_7592.JPG, Category: spindle, Score: 11.6%
File: IMG_7976.JPG, Category: tiger cat, Score: 17.2%
File: IMG_8170.JPG, Category: lion, Score: 39.3%
File: IMG_9454.JPG, Category: tabby, Score: 35.2%
File: IMG_3752.JPG, Category: Persian cat, Score: 36.1%
File: IMG_5013_polarr.jpg, Category: bathtub, Score: 14.1%
File: IMG_9458.JPG, Category: Siamese cat, Score: 8.0%
File: IMG_4641_polarr.JPG, Category: Siamese cat, Score: 88.8%
File: IMG_6126.JPG, Category: Persian cat, Score: 26.6%
File: IMG_8877.JPG, Category: Cardigan, Score: 17.1%


Your answer:

~~~
Please write your answer in full sentences.


~~~